In [ ]:
import torch.nn as nn
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

from tqdm import tqdm

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)


class Generator(nn.Module):
    def __init__(self, ):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        # input data는 [batch size, 100, 1, 1]의 형태로 주어야합니다.
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self, d=64):
      super(Discriminator, self).__init__()

      self.layer1 = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=d, kernel_size=4, stride=2, padding=1),
          nn.BatchNorm2d(d),
          nn.LeakyReLU(0.2, inplace=True),
      )

      self.layer2 = nn.Sequential(
          nn.Conv2d(in_channels=d, out_channels=d*2, kernel_size=4, stride=2, padding=1),
          nn.BatchNorm2d(d*2),
          nn.LeakyReLU(0.2, inplace=True),
      )

      self.layer3 = nn.Sequential(
          nn.Conv2d(in_channels=d*2, out_channels=d*4, kernel_size=4, stride=2, padding=1),
          nn.BatchNorm2d(d*4),
          nn.LeakyReLU(0.2, inplace=True),
      )

      self.layer4 = nn.Sequential(
          nn.Conv2d(in_channels=d*4, out_channels=d*8, kernel_size=4, stride=2, padding=1),
          nn.BatchNorm2d(d*8),
          nn.LeakyReLU(0.2, inplace=True),
      )

      self.layer5 = nn.Sequential(
          nn.Conv2d(in_channels=d*8, out_channels=1, kernel_size=4, stride=1, padding=0),
          nn.Sigmoid()
      )
    
    def forward(self, input):
      input = self.layer1(input)
      input = self.layer2(input)
      input = self.layer3(input)
      input = self.layer4(input)
      input = self.layer5(input)
      return input


if __name__ == "__main__":
    # 학습코드는 모두 여기서 작성해주세요

    data_path = 'training_data/'

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5079152,0.42205644,0.37666804],[0.25580716, 0.23393774, 0.23002408])
    ])

    dataset = datasets.ImageFolder(root=data_path,
                                   transform=transform,
                                   )

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    lr = 0.0002
    batch_size = 128

    train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    generator = Generator().to(device)
    discriminator = Discriminator().to(device)

    criterion = torch.nn.BCELoss()
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)

    epochs = 30
    total_batch_num = len(train_dataloader)

    for epoch in range(epochs):
      generator.train()
      discriminator.train()

      avg_g_cost = 0
      avg_d_cost = 0

      for step, batch in enumerate(train_dataloader):
        b_x = batch[0]
        num_img = len(b_x)

        real_label = torch.ones((num_img, 1, 1, 1)).to(device)
        fake_label = torch.zeros((num_img, 1, 1, 1)).to(device)

        real_logit = discriminator(b_x.to(device))
        d_real_loss = criterion(real_logit, real_label)

        z = torch.randn((num_img, 100, 1, 1), requires_grad = False).to(device)
        fake_data = generator(z)
        fake_logit = discriminator(fake_data)
        d_fake_loss = criterion(fake_logit, fake_label)
        
        d_loss = d_real_loss + d_fake_loss
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        z = torch.randn((num_img, 100, 1, 1), requires_grad = False).to(device)
        fake_data = generator(z)
        fake_logit = discriminator(fake_data)
        g_loss = criterion(fake_logit, real_label)

        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        avg_d_cost += d_loss
        avg_g_cost += g_loss

      avg_d_cost /= total_batch_num
      avg_g_cost /= total_batch_num

      print("Epoch: {}/{}, discriminator cost: {}, generator cost:{}".format(epoch+1, epochs, avg_d_cost, avg_g_cost))

    # FID score 측정에 사용할 fake 이미지를 생성하는 코드 입니다.
    # generator의 학습을 완료한 뒤 마지막에 실행하여 fake 이미지를 저장하시기 바랍니다.
    test_noise = torch.randn(3000, 100, 1, 1, device=device)
    with torch.no_grad():
        test_fake = generator(test_noise).detach().cpu()

        for index, img in enumerate(test_fake):
            fake = np.transpose(img.detach().cpu().numpy(), [1, 2, 0])
            fake = (fake * 127.5 + 127.5).astype(np.uint8)
            im = Image.fromarray(fake)
            im.save("./fake_img/fake_sample{}.jpeg".format(index))

In [ ]:
import os
import torch

from pytorch_fid.fid_score import *

os.environ['KMP_DUPLICATE_LIB_OK']='True'

real_img_path = 'training_data/celeba/'
fake_img_path = 'fake_img/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if __name__ == "__main__":
    fid = calculate_fid_given_paths(
        paths=[real_img_path, fake_img_path],
        batch_size=128,
        device=device,
        dims=2048
    )

    print("fid score : {}".format(fid))
